# json ファイルの基本的な扱い方

In [ ]:
import json
from datetime import date
from pathlib import Path

In [ ]:
json_path = Path.cwd().parent.joinpath("data").joinpath("sample.json")

# data/sample.json を読み込む。
# create_json.py で生成したファイルを開く。
# Windows では、UTF-8 の BOM が付与されている可能性があるため、encoding="utf_8_sig" を指定する。
with json_path.open("r", encoding="utf_8_sig") as f:
    # 読み込んだファイルを data 変数に格納する。
    data = json.load(f)

# data には辞書型のデータが格納されている。
data

## 辞書型

In [ ]:
# メタデータを参照する。
data["metadata"]

In [ ]:
# get メソッドでも同じことができる。
# 一つ上の方法よりもほんの僅かに遅いが、エラーが発生しない。
# 速度低下は無視できる程度である。
# どちらを使うかはケースバイケースで判断する。正直どっちでもいいと思ってる。
data.get("metadata")

In [ ]:
# 参照する名前が存在しない場合は KeyError が発生する。
data["not_exist"]

In [ ]:
# get メソッドを使うと、キーが存在しない場合に指定した値を返す。
# 2 番目の引数は省略可能で、省略した場合は None が返る。
data.get("not_exist", "default")

In [ ]:
# metadata の中身も同様に参照できる。
data["metadata"]["version"]

In [ ]:
# get メソッドでももちろん可能。
data.get("metadata").get("version")

## リスト型

In [ ]:
# ユーザ情報はリスト型で格納されている。
data["users"]

In [ ]:
# 1 つ目を参照する。
# 番号が 0 から始まることに注意。
data["users"][0]

In [ ]:
# 最後から 1 つ目を参照する。
# マイナス n で最後から n 番目を参照できる。
data["users"][-1]

In [ ]:
# 範囲外のインデックスを指定すると IndexError が発生する。
data["users"][100_000_000]

In [ ]:
# いちいち data["users"] と書くのが面倒な場合は、変数に格納しておくと便利。
users = data["users"]
users[0]

In [ ]:
# リストの中身も辞書型なので、辞書型と同じように参照できる。
users[0]["name"], users[0].get("birthday"), users[0].get("address")

In [ ]:
# len 関数でリストの長さを取得できる。
len(users)

In [ ]:
# 名前だけを取り出したいときは、リスト内包表記を使う。
# リスト内包表記はリストの中身を一括で処理するための構文。
# この場合、users の要素を一つずつ user に格納し、
# user["name"] を取り出したものだけをリストに格納する。
names = [user["name"] for user in users]
names

In [ ]:
# リスト内包表記は for 文を使った書き方と同じ意味を持つ。
# 筆者はリスト内包が好き。
# Python で append するのは遅いので、リスト内包表記を使うと速いのでおすすめ。
names = []
for user in users:
    names.append(user["name"])
names

## フィルタリング

In [ ]:
# フィルタリングの準備として、誕生日を Python で扱いやすい date 型に変換する。
# 辞書型もリスト型も、中身を = で更新することができる。
for user in users:
    user["birthday"] = date.fromisoformat(user["birthday"])

# birthday の型が変わっていることを確認する。
users[0]

In [ ]:
# リスト内包表記は if 文を使ったフィルタリングもできる。
# この場合、user["birthday"] が 2000 年以降のユーザだけを取り出す。
selected_users = [user for user in users if user.get("birthday") >= date(2000, 1, 1)]
selected_users

In [ ]:
# for 文を使った書き方と同じ意味を持つ。
selected_users = []
for user in users:
    if user.get("birthday") >= date(2000, 1, 1):
        selected_users.append(user)
selected_users

## json ファイルの書き込み

In [ ]:
# json ファイルに書き込む。
# date 型は json モジュールでは扱えないため、元に戻す。
for user in selected_users:
    user["birthday"] = user["birthday"].isoformat()
# 読み込んだファイルと同じディレクトリの output.json に書き込む。
output_path = json_path.parent.joinpath("output.json")
with output_path.open("w", encoding="utf_8") as f:
    # この場合、出力ファイルは整形されていない。
    json.dump(selected_users, f)
# 整形したい場合は、indent 引数を指定する。
output_path = json_path.parent.joinpath("output_pretty.json")
with output_path.open("w", encoding="utf_8") as f:
    json.dump(selected_users, f, indent=4)
# 仮に、日本語が含まれている場合は、ensure_ascii=False を指定すると見やすくなる。
output_path = json_path.parent.joinpath("output_japanese.json")
with output_path.open("w", encoding="utf_8") as f:
    json.dump(selected_users, f, indent=4, ensure_ascii=False)

## 時間計測

```
CPU: AMD Ryzen 9 5900X
Python: 3.13.0
OS: Windows 11
SSD: 知らん
```

In [ ]:
%%timeit
# 読み込みにかかる時間を計測する。

with json_path.open("r", encoding="utf_8_sig") as f:
    # 読み込んだファイルを data 変数に格納する。
    data = json.load(f)
# birthday の型を変換する。
for user in data["users"]:
    user["birthday"] = date.fromisoformat(user["birthday"])

In [ ]:
with json_path.open("r", encoding="utf_8_sig") as f:
    # 読み込んだファイルを data 変数に格納する。
    data = json.load(f)
# birthday の型を変換する。
for user in data["users"]:
    user["birthday"] = date.fromisoformat(user["birthday"])

In [ ]:
%%timeit
# birthday でフィルタリングする。
selected_users = [user for user in data["users"] if user.get("birthday") < date(2000, 1, 1)]